<pre>
Table: Visits

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| visit_id    | int     |
| customer_id | int     |
+-------------+---------+
visit_id is the column with unique values for this table.
This table contains information about the customers who visited the mall.
 

Table: Transactions

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| transaction_id | int     |
| visit_id       | int     |
| amount         | int     |
+----------------+---------+
transaction_id is column with unique values for this table.
This table contains information about the transactions made during the visit_id.
 

Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

Return the result table sorted in any order.

The result format is in the following example.

 

Example 1:

Input: 
Visits
+----------+-------------+
| visit_id | customer_id |
+----------+-------------+
| 1        | 23          |
| 2        | 9           |
| 4        | 30          |
| 5        | 54          |
| 6        | 96          |
| 7        | 54          |
| 8        | 54          |
+----------+-------------+
Transactions
+----------------+----------+--------+
| transaction_id | visit_id | amount |
+----------------+----------+--------+
| 2              | 5        | 310    |
| 3              | 5        | 300    |
| 9              | 5        | 200    |
| 12             | 1        | 910    |
| 13             | 2        | 970    |
+----------------+----------+--------+
Output: 
+-------------+----------------+
| customer_id | count_no_trans |
+-------------+----------------+
| 54          | 2              |
| 30          | 1              |
| 96          | 1              |
+-------------+----------------+
Explanation: 
Customer with id = 23 visited the mall once and made one transaction during the visit with id = 12.
Customer with id = 9 visited the mall once and made one transaction during the visit with id = 13.
Customer with id = 30 visited the mall once and did not make any transactions.
Customer with id = 54 visited the mall three times. During 2 visits they did not make any transactions, and during one visit they made 3 transactions.
Customer with id = 96 visited the mall once and did not make any transactions.
As we can see, users with IDs 30 and 96 visited the mall one time without making any transactions. Also, user 54 visited the mall twice and did not make any transactions.
</pre>

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# importing pyspark sql functions
from pyspark.sql.functions import *
from pyspark.sql.window import *

# importing sql types from pyspark
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, IntegerType, DateType, TimestampType

# importing SparkSession
from pyspark.sql import SparkSession

from datetime import datetime


In [0]:
# creating spark session and providing app name
spark = SparkSession.builder.appName("leetcode-top-50-sql-solution-with-pyspark").getOrCreate()

In [0]:
# creating Schema
# Define the schema for the Weather table
weather_schema = StructType([
    StructField("id", IntegerType(), nullable=False),
    StructField("recordDate", DateType(), nullable=False),
    StructField("temperature", IntegerType(), nullable=False)
])


In [0]:

weather_df = spark.createDataFrame([
     (1, datetime.strptime("2015-01-01", "%Y-%m-%d").date(), 10),
    (2, datetime.strptime("2015-01-02", "%Y-%m-%d").date(), 25),
    (3, datetime.strptime("2015-01-03", "%Y-%m-%d").date(), 20),
    (4, datetime.strptime("2015-01-04", "%Y-%m-%d").date(), 30)
], schema=weather_schema)






In [0]:
weather_df.display()

id,recordDate,temperature
1,2015-01-01,10
2,2015-01-02,25
3,2015-01-03,20
4,2015-01-04,30


In [0]:
# Leetcode Solution in Spark SQL
# Creating Temporary view for the product dataframe for sql queries
weather_df.createOrReplaceTempView('weather')
sql_result = spark.sql(
    '''
    SELECT 
    tab_one.id
    FROM weather AS tab_one
    JOIN weather AS tab_two
    ON datediff(tab_one.recordDate, tab_two.recordDate) = 1
    WHERE tab_one.temperature > tab_two.temperature;
    
    '''
)

# Displaying Result
sql_result.display()

id
2
4


In [0]:
# Leet Code Solution in Data Frame

# Creating Windo Spec ordering the data by recordDate
order_tab = Window.orderBy("recordDate")

# Creating a seperate column for previous temperature
weather_df = weather_df.withColumn("prev_temp", lag("temperature").over(order_tab))

# Filtering only id from rows where actual temperature is greater than previous temperature
# Displaying the filtered Result
filter_result = weather_df.filter(col('temperature') > col('prev_temp')).select('id').display()

id
2
4
